In [1]:
%pip install bioinfokit

import pandas as pd
import scipy.stats
import os
from bioinfokit import analys, visuz
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.stats.multitest import multipletests
import logging
import statsmodels
import matplotlib


import platform


print("\nPackage versions used:")
print(f"Python: {platform.python_version()}")
print(f"Pandas: {pd.__version__}")
print(f"Numpy: {np.__version__}")
print(f"SciPy: {scipy.__version__}")
print(f"Statsmodels: {statsmodels.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")


# Setup basic logging configuration
logging.basicConfig(
    filename="processing.log",
    filemode="w",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

Note: you may need to restart the kernel to use updated packages.

Package versions used:
Python: 3.11.11
Pandas: 2.3.0
Numpy: 2.2.6
SciPy: 1.16.0
Statsmodels: 0.14.5
Matplotlib: 3.10.3
Seaborn: 0.13.2


In [4]:
def list_dataset_files(path: str) -> list:
    """List all .tsv files in the dataset directory."""
    return sorted([f for f in os.listdir(path) if f.endswith(".tsv")])

def load_and_parse_file(file_path: str) -> pd.DataFrame:
    """Load the TSV file into a pandas DataFrame."""
    df = pd.read_csv(file_path, sep='\t')
    pd.set_option('display.float_format', '{:.6g}'.format)
    return df

def expand_gene_symbols(df: pd.DataFrame) -> pd.DataFrame:
    """Expand gene symbols split by '///' into separate rows. Supports flexible column names."""
    possible_columns = ['GENE_SYMBOL', 'Gene.symbol', 'Gene.title']
    symbol_col = None

    for col in possible_columns:
        if col in df.columns:
            symbol_col = col
            break

    if symbol_col is None:
        raise ValueError("No known gene symbol column found. Checked: " + ", ".join(possible_columns))

    df2 = pd.DataFrame(columns=['GENE_SYMBOL', 'logFC', 'P.Value'])
    notincluded = []

    for index, row in df.iterrows():
        try:
            for item in str(row[symbol_col]).split('///'):
                df2.loc[len(df2.index)] = [item.strip(), row["logFC"], row["P.Value"]]
        except:
            notincluded.append(row.get('ID', index))

    return df2

def combine_duplicate_genes(df: pd.DataFrame) -> pd.DataFrame:
    """Average logFC and combine p-values using Fisher's method for duplicated genes."""
    def average(lst):
        return sum(lst) / len(lst)

    gene_dict = {}
    count = {}
    for gene in df['GENE_SYMBOL']:
        if gene not in count:
            gene_dict[gene] = {'logFC': [], 'P.Value': []}
            count[gene] = 1
        else:
            count[gene] += 1

    for k in list(count.keys()):
        if count[k] == 1:
            del gene_dict[k]

    df_clean = df.copy()
    for index, row in df.iterrows():
        gene = row['GENE_SYMBOL']
        if gene in gene_dict:
            gene_dict[gene]['logFC'].append(row['logFC'])
            gene_dict[gene]['P.Value'].append(row['P.Value'])
            df_clean.drop(index, inplace=True)

    for gene, values in gene_dict.items():
        _, combined_p = scipy.stats.combine_pvalues(values['P.Value'], method='fisher')
        mean_logfc = average(values['logFC'])
        df_clean = pd.concat([df_clean, pd.DataFrame([{
            'GENE_SYMBOL': gene,
            'logFC': mean_logfc,
            'P.Value': combined_p
        }])], ignore_index=True)

    return df_clean.sort_values(by="P.Value")

def adjust_pvalues(df: pd.DataFrame, pval_col: str = 'P.Value') -> pd.DataFrame:
    """Add Benjamini-Hochberg FDR correction to the dataframe."""
    corrected = multipletests(df[pval_col], method='fdr_bh')
    df['FDR'] = corrected[1]
    return df

def plot_all(df: pd.DataFrame, output_prefix: str,
             lfc_col='logFC', fdr_col='FDR',
             lfc_thresh=0.2, fdr_thresh=0.05):
    """Generate volcano, boxplot, and FDR histogram for a dataset."""

    # Volcano plot using FDR
    df['-log10(FDR)'] = -np.log10(df[fdr_col].replace(0, np.nextafter(0, 1)))  # avoid -inf
    df['significant'] = (abs(df[lfc_col]) > lfc_thresh) & (df[fdr_col] < fdr_thresh)

    plt.figure(figsize=(8, 6))
    plt.scatter(df[lfc_col], df['-log10(FDR)'],
                c=df['significant'].map({True: 'red', False: 'grey'}),
                alpha=0.7, edgecolors='k')
    plt.axhline(-np.log10(fdr_thresh), linestyle='--', color='blue')
    plt.axvline(-lfc_thresh, linestyle='--', color='blue')
    plt.axvline(lfc_thresh, linestyle='--', color='blue')
    plt.title("Volcano Plot (FDR-based)")
    plt.xlabel("log2 Fold Change")
    plt.ylabel("-log10(FDR)")
    plt.tight_layout()
    plt.savefig(output_prefix + "_volcano.png", dpi=300)
    plt.close()

    # Boxplot of logFC
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[lfc_col])
    plt.title("logFC Distribution")
    plt.tight_layout()
    plt.savefig(output_prefix + "_logFC_boxplot.png", dpi=300)
    plt.close()

    # Histogram of FDR values
    plt.figure(figsize=(6, 4))
    plt.hist(df[fdr_col], bins=50, color='skyblue', edgecolor='black')
    plt.title("FDR Distribution")
    plt.xlabel("FDR")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(output_prefix + "_fdr_histogram.png", dpi=300)
    plt.close()


def process_dataset(data_path: str, file: str, results_root: str) -> pd.DataFrame:
    dataset_name = os.path.basename(data_path)
    output_dir = os.path.join(results_root, dataset_name)
    os.makedirs(output_dir, exist_ok=True)

    file_path = os.path.join(data_path, file)
    output_prefix = os.path.join(output_dir, file.replace(".tsv", ""))

    logging.info(f"Start processing file: {file} in dataset: {dataset_name}")
    print(f"→ Processing {dataset_name}/{file}")
    try:
        df = pd.read_csv(file_path, sep='\t')
        pd.set_option('display.float_format', '{:.6g}'.format)
        initial_gene_count = len(df)

        # Expand gene symbols with fallback column detection
        possible_columns = ['GENE_SYMBOL', 'Gene.symbol', 'Gene.title']
        symbol_col = next((col for col in possible_columns if col in df.columns), None)
        if not symbol_col:
            raise ValueError(f"No gene symbol column found in {file}")

        df2 = pd.DataFrame(columns=['GENE_SYMBOL', 'logFC', 'P.Value'])
        for index, row in df.iterrows():
            try:
                for item in str(row[symbol_col]).split('///'):
                    df2.loc[len(df2.index)] = [item.strip(), row["logFC"], row["P.Value"]]
            except:
                continue

        expanded_count = len(df2)
        unique_genes = df2['GENE_SYMBOL'].nunique()
        duplicates = expanded_count - unique_genes

        # Combine duplicates
        gene_dict = {}
        for gene in df2['GENE_SYMBOL'].unique():
            subset = df2[df2['GENE_SYMBOL'] == gene]
            if len(subset) > 1:
                logfc_avg = subset['logFC'].astype(float).mean()
                _, p_comb = scipy.stats.combine_pvalues(subset['P.Value'].astype(float), method='fisher')
            else:
                logfc_avg = subset['logFC'].values[0]
                p_comb = subset['P.Value'].values[0]
            gene_dict[gene] = (logfc_avg, p_comb)

        df_combined = pd.DataFrame([(k, v[0], v[1]) for k, v in gene_dict.items()],
                                   columns=['GENE_SYMBOL', 'logFC', 'P.Value'])

        # Adjust p-values
        df_combined['FDR'] = multipletests(df_combined['P.Value'], method='fdr_bh')[1]
        cleaned_count = len(df_combined)

        # Count significant genes
        significant_count = (df_combined['FDR'] < 0.05).sum()

        # Output
        output_file = os.path.join(output_dir, file.replace(".tsv", "_processed.csv"))
        df_combined.to_csv(output_file, index=False)

        stats_msg = (
            f"{dataset_name}/{file}:\n"
            f"  Original genes: {initial_gene_count}\n"
            f"  Expanded rows: {expanded_count}\n"
            f"  Unique genes before deduplication: {unique_genes}\n"
            f"  Duplicated genes: {duplicates}\n"
            f"  Final unique genes after combining: {cleaned_count}\n"
            f"  Genes with FDR < 0.05: {significant_count}\n"
        )
        print(stats_msg)
        logging.info(stats_msg)

        return df_combined

    except Exception as e:
        error_msg = f"Failed to process {file} in {dataset_name}: {e}"
        logging.error(error_msg)
        print(error_msg)
        return pd.DataFrame()  # return empty dataframe on failure

def batch_process_all_datasets(data_root: str, results_root: str = "../results") -> dict:
    """Process all datasets and save to results_root with logging and printed stats."""
    os.makedirs(results_root, exist_ok=True)
    results = {}
    dataset_dirs = [d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))]

    for dataset_name in dataset_dirs:
        dataset_path = os.path.join(data_root, dataset_name)
        files = list_dataset_files(dataset_path)

        for file in files:
            df_processed = process_dataset(dataset_path, file, results_root)
            results[f"{dataset_name}/{file}"] = df_processed

    return results

In [5]:
batch_process_all_datasets("../data", "../results")


→ Processing GSE116280/GSE116280_Rotenone_100nm_24h_8d.tsv
GSE116280/GSE116280_Rotenone_100nm_24h_8d.tsv:
  Original genes: 21727
  Expanded rows: 21727
  Unique genes before deduplication: 21256
  Duplicated genes: 471
  Final unique genes after combining: 21256
  Genes with FDR < 0.05: 1680

→ Processing GSE116280/GSE116280_Rotenone_50nm_12h_8d.tsv
GSE116280/GSE116280_Rotenone_50nm_12h_8d.tsv:
  Original genes: 21728
  Expanded rows: 21728
  Unique genes before deduplication: 21257
  Duplicated genes: 471
  Final unique genes after combining: 21257
  Genes with FDR < 0.05: 318

→ Processing GSE116280/GSE116280_Rotenone_50nm_24h_8d.tsv
GSE116280/GSE116280_Rotenone_50nm_24h_8d.tsv:
  Original genes: 21729
  Expanded rows: 21729
  Unique genes before deduplication: 21258
  Duplicated genes: 471
  Final unique genes after combining: 21258
  Genes with FDR < 0.05: 1204

→ Processing GSE90122/GSE90122_Rifampicin.tsv
GSE90122/GSE90122_Rifampicin.tsv:
  Original genes: 42545
  Expanded rows:

{'GSE116280/GSE116280_Rotenone_100nm_24h_8d.tsv':        GENE_SYMBOL     logFC  P.Value        FDR
 0         H1FX-AS1  -1.86922 1.42e-07 0.00150918
 1             PENK  -2.18635 5.51e-07 0.00369854
 2             DARC  -1.55903  8.5e-07 0.00369854
 3           PRSS56   1.78348  8.7e-07 0.00369854
 4        LOC388882  -1.15597 1.52e-06 0.00477294
 ...            ...       ...      ...        ...
 21251         PIRT -6.35e-05    0.999   0.999188
 21252  XLOC_007703 0.0001281        1          1
 21253       TCERG1 -5.58e-05        1          1
 21254        DDX58  2.64e-05        1          1
 21255     SNORA16B  2.18e-05        1          1
 
 [21256 rows x 4 columns],
 'GSE116280/GSE116280_Rotenone_50nm_12h_8d.tsv':       GENE_SYMBOL     logFC  P.Value        FDR
 0        H1FX-AS1  -1.98436 1.77e-07 0.00188124
 1           TXNIP  -2.51847 4.07e-07 0.00288387
 2          ARRDC4  -1.92979 1.01e-06 0.00536739
 3          BDKRB2   1.58583 1.34e-06 0.00569688
 4            FAT1   1.65608 